In [1]:
DEBUG = False

In [2]:
import numpy as np
import pandas as pd
import os, random, glob, cv2, gc
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from termcolor import colored
import skimage.io
import sys

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip, RandomSizedCrop, Rotate, Transpose

from functools import partial
import scipy as sp
from zipfile import ZipFile
from PIL import Image
import time

In [3]:
class config:
    IMG_SIZE = 512
    BS = 8
    SZ = 256
    N = 36
    LEVEL = 1
    SEED = 2020
    LR = 0.0001
    LOG = './train_log.txt'

In [4]:
COMP_DIR = '../input/prostate-cancer-grade-assessment/'
TRAIN_DIR = COMP_DIR + 'train_images/'

In [5]:
# seed
def seed_everything(seed=2020):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(config.SEED)


In [6]:
torch.cuda.empty_cache()
gc.collect()

60

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# Data

In [8]:
df = pd.read_csv(COMP_DIR + 'train.csv').set_index('image_id')
df = df.iloc[:100] if DEBUG else df

In [9]:
# shuffle
df = shuffle(df)

# split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=config.SEED)
val_df, test_df = train_test_split(val_df, test_size=0.2, random_state=config.SEED)

train_df = train_df.reset_index()
val_df = val_df.reset_index()
test_df = test_df.reset_index()

print(len(train_df), len(val_df), len(test_df))

8492 1699 425


In [10]:
train_df.head()

,image_id,data_provider,isup_grade,gleason_score
0,664a26f04c363ae643886559d5910e16,karolinska,0,0+0
1,5a599e2b3cb03e9525de22d1c27f4ed5,karolinska,1,3+3
2,3bb7fa03be7da5a086cf9a234cd3a025,karolinska,4,4+4
3,673fea26c4fa022353f635b89d6fd4b2,radboud,3,4+3
4,ae9c53b32653368de19c3433eb91b6d9,karolinska,0,0+0


# Augmentation

In [11]:
train_transforms = Compose([Transpose(p=0.5),
                            HorizontalFlip(p=0.5), 
                            VerticalFlip(p=0.5),
                            Rotate(p=0.3)])
val_transforms = Compose([])

# Dataset

In [12]:
def get_tiles(path, level=1, SZ=256, N=36, mode=0):

    img = skimage.io.MultiImage(path)[level]
    
    h, w, c = img.shape
    pad_h = (SZ - h % SZ) % SZ + ((SZ * mode) // 2)
    pad_w = (SZ - w % SZ) % SZ + ((SZ * mode) // 2)

    img2 = np.pad(img, [[pad_h // 2, pad_h - pad_h // 2], 
                        [pad_w // 2, pad_w - pad_w//2], 
                        [0,0]], constant_values=255)
    img3 = img2.reshape(
        img2.shape[0] // SZ,
        SZ,
        img2.shape[1] // SZ,
        SZ,
        3
    )
    
    img3 = img3.transpose(0, 2, 1, 3, 4).reshape(-1, SZ, SZ, 3) # (783, 256, 256, 3)

    # supplement
    if len(img3) < N:
        img3 = np.pad(img3, [ [0, N-len(img3)], [0,0],[0,0],[0,0]], constant_values=255)
        
    idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:N]
    img3 = img3[idxs]
    
    return img3

In [13]:
class MyDataset(Dataset):
    def __init__(self, df, split='train', shuffle_df=False, shuffle_tiles=False):
        super().__init__()
        
        if shuffle_df:
            df = shuffle(df)
        self.df = df.reset_index(drop=True)
        
        self.split = split
        self.shuffle_tiles = shuffle_tiles
            
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        name = self.df.image_id[idx] + '.tiff'
            
        # get tiles
        tiles = get_tiles(TRAIN_DIR + name, config.LEVEL, config.SZ, config.N)
        
        # apply transform to each img
        imgs = []
        for t in tiles:
            if self.split == 'train':
                t_aug = train_transforms(**{'image': t})['image']
            elif self.split == 'val':
                t_aug = val_transforms(**{'image': t})['image']
                
            imgs.append(t_aug) # already tensor
        
        
        # shuffle tiles
        if self.shuffle_tiles:
            imgs = shuffle(imgs)
            
        
        
        # concat
        n = int(np.sqrt(config.N))
        images = np.zeros((config.SZ*n, config.SZ*n, 3), dtype=np.int32)
        for i in range(n):
            for j in range(n):
                images[i*config.SZ : (i+1)*config.SZ, j*config.SZ : (j+1)*config.SZ, :] = imgs[i*n+j]
             
            
        
        # normalize
        images = 255 - images # reverse 
        images = images / 255. # [0, 1]
        
        
        
        # resize 
        images = cv2.resize(images, (config.IMG_SIZE, config.IMG_SIZE))
        images = torch.tensor(images).permute(2, 0, 1)
        label = torch.tensor(self.df.isup_grade[idx])
        return images, label
        

In [14]:
# test dataset
if DEBUG:
    train_ds = MyDataset(train_df, 'train')
    x, y = train_ds[6]
    print(x.shape)
    plt.imshow(x.permute(1, 2, 0).numpy())
    plt.title(y)
    
    del train_ds, x, y

In [15]:
train_ds = MyDataset(train_df, 'train')
train_dl = DataLoader(train_ds, batch_size=config.BS, shuffle=True, drop_last=True) # use 32 means 2 images per batch, don't shuffle to preserve 

val_ds = MyDataset(val_df, 'val')
val_dl = DataLoader(val_ds, batch_size=config.BS, shuffle=False, drop_last=True)

test_ds = MyDataset(test_df, 'val')
test_dl = DataLoader(test_ds, batch_size=config.BS, shuffle=False, drop_last=True)

In [16]:
# test dl
if DEBUG:
    x, y = next(iter(train_dl))
    plt.imshow(x[0].permute(1, 2, 0).numpy())
    plt.title(y[0])

# Model

In [17]:
sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

In [18]:
from efficientnet_pytorch import EfficientNet

class MyModel(nn.Module):
    def __init__(self, backbone='efficientnet-b4'):
        super().__init__()
        
        self.base = EfficientNet.from_name(backbone)
        self.base.load_state_dict(torch.load('../input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth'))
        self.fc = nn.Linear(self.base._fc.in_features, 1)
        self.base._fc = nn.Identity()
    
    def forward(self, x):
        x = self.base(x)
        x = self.fc(x)
        return x

In [19]:
if DEBUG:
    model = MyModel()
    x = torch.randn(2, 3, 512, 512) # for b4, it can only accepts 512
    y = model(x)
    print(y.shape)

In [20]:
model = MyModel()
model = model.to(device)
model.load_state_dict(torch.load('../input/pandaefnb4regression/efnb4.pth'))

optimizer = optim.Adam(model.parameters(), lr=config.LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=5, verbose=1)

In [21]:
# freeze 
# def freeze_until(model, name):
#     flag = False
#     for n, p in model.named_parameters():
#         if n == name:
#             flag = True
#         p.requires_grad = flag
# freeze_until(model, 'model.layer4.2.conv1.weight')

# Training

In [22]:
class OptimizedRounder():
    def __init__(self):
        self.coef_ = [0.5, 1.5, 2.5, 3.5, 4.5]

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 4
            else:
                X_p[i] = 5

        ll = cohen_kappa_score(y, X_p, weights='quadratic')

        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5, 4.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef=[0.5, 1.5, 2.5, 3.5, 4.5]):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 4
            else:
                X_p[i] = 5
        return X_p

    def coefficients(self):
        '''use after self.fit or error throws'''
        return self.coef_['x']

In [23]:
rounder = OptimizedRounder()

In [24]:
def train_on(epoch):
    
    torch.cuda.empty_cache()
    gc.collect()
    
    model.train()
    
    loss_epoch = []
    preds_epoch = []
    y_epoch = []
    bar = tqdm(enumerate(train_dl), total=len(train_dl))
    for i, (x, y) in bar:
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        y_preds = model(x) # [batch, 1]
        y_preds = y_preds.view(-1) #[batch,]
        
        # get metrics
        loss = nn.MSELoss()(y_preds, y)

        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # add
        y_np = y.cpu().detach().numpy()
        y_preds_np = rounder.predict(y_preds.cpu().detach().numpy())
        loss_np = loss.cpu().detach().numpy()
        
        # add
        preds_epoch.append(y_preds_np)
        y_epoch.append(y_np)
        loss_epoch.append(loss_np)
    
        
        c = cohen_kappa_score(y_np, y_preds_np, weights='quadratic')
        bar.set_description('loss: %.4f, cohen: %.4f' % (loss_np, c))
        
            
        # clean
        del x, y, y_preds, loss, y_np, y_preds_np, loss_np
        torch.cuda.empty_cache()
        gc.collect()
        
        
    cohen = cohen_kappa_score(np.concatenate(preds_epoch), 
                              np.concatenate(y_epoch), 
                              weights='quadratic')
    
    print('Epoch: %d, Loss: %.4f, Cohen: %.4f' % (epoch, np.mean(loss_epoch), cohen))
    return np.mean(loss_epoch), cohen


In [25]:
torch.cuda.empty_cache()
gc.collect()

60

In [26]:
if DEBUG:
    train_on(1)

In [27]:
def val_on(epoch, dl):

    torch.cuda.empty_cache()
    gc.collect()
    
    model.eval()
    
    loss_epoch = []
    preds_epoch = []
    y_epoch = []
    bar = tqdm(enumerate(dl), total=len(dl))
    with torch.no_grad():
        for i, (x, y) in bar:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)
            
            y_preds = model(x).view(-1)

            # get metrics
            loss = nn.MSELoss()(y_preds, y)

            # add
            y_preds_np = y_preds.cpu().detach().numpy()
            y_np = y.cpu().detach().numpy()
            loss_np = loss.cpu().detach().numpy()
            
            loss_epoch.append(loss_np)
            preds_epoch.append(y_preds_np)
            y_epoch.append(y_np)
            
            
            # get cohen
            c = cohen_kappa_score(rounder.predict(y_preds_np), y_np, weights='quadratic')
            bar.set_description('loss: %.4f, cohen: %.4f' % (loss_np, c))

        
            # clean
            del x, y, y_preds, loss, y_np, y_preds_np, loss_np, c
            torch.cuda.empty_cache()
            gc.collect()
            

    # cohen
    preds_epoch = np.concatenate(preds_epoch)
    y_epoch = np.concatenate(y_epoch)
    
    rounder.fit(preds_epoch, y_epoch)
    coef = rounder.coefficients()
    preds_epoch = rounder.predict(preds_epoch, coef)
    cohen = cohen_kappa_score(preds_epoch, y_epoch, weights='quadratic')
    
    print('Epoch: %d, Loss: %.4f, Cohen: %.4f' % (epoch, np.mean(loss_epoch), cohen))

    return np.mean(loss_epoch), cohen, coef
        
            

In [28]:
if DEBUG:
    val_on(1, val_dl)

In [29]:
def train(epochs):
    best_cohen = []
    for e in range(epochs):
        train_loss, train_cohen = train_on(e)
        val_loss, val_cohen, coef = val_on(e, val_dl)
        
        # adjust lr
        scheduler.step(val_loss)

        # write to log
        with open(config.LOG, 'a') as f:
            f.write(time.ctime() + f' Epoch: {e}, Train loss: {(train_loss):.4f}, Train cohen: {(train_cohen):.4f}\n   Val loss: {(val_loss):.4f}, Val cohen:{(val_cohen):.4f}, Coef: {coef} \n\n')
            
        # save best
        best_cohen.append(val_cohen)
        if val_cohen >= max(best_cohen):
            print('save best model')
            torch.save(model.state_dict(), f'seresnetxt-best.pth')
            
    torch.save(model.state_dict(), './final.pth')

In [30]:
train(3)


Epoch: 0, Loss: 0.4450, Cohen: 0.9068



Epoch: 0, Loss: 1.0471, Cohen: 0.8105
save best model



Epoch: 1, Loss: 0.3714, Cohen: 0.9225



Epoch: 1, Loss: 0.9570, Cohen: 0.8223
save best model



Epoch: 2, Loss: 0.3323, Cohen: 0.9312



Epoch: 2, Loss: 1.0190, Cohen: 0.8171


In [32]:
val_on(1, test_dl)


Epoch: 1, Loss: 1.1641, Cohen: 0.7975


(1.16406,
 0.7975275438945004,
 array([0.51484739, 1.502021  , 2.41180707, 3.46734047, 4.65588263]))